The Xarray homepage is found here: http://xarray.pydata.org/

In [72]:
import xarray as xr
import numpy as np
import datetime
%matplotlib inline

In [4]:
ds = xr.open_dataset('/Users/leo/Dropbox/education/SOAC/2017/SOAC_shared/data/Gomez/precip.2013.Gomez.nc')
#ds.info()

In [10]:
ds.groupby('time.month').mean()

<xarray.Dataset>
Dimensions:    (month: 12)
Coordinates:
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    time_bnds  (month) float64 2.303e+04 2.306e+04 2.309e+04 2.312e+04 ...
    precip     (month) float64 0.1227 0.5273 0.4347 0.3978 0.1705 0.1424 ...

In [5]:
x = (ds['time.season'] == 'DJF').values
#ix = np.where(x)[0]
ix, = x.nonzero()
#ixr = xr.DataArray(ix,dims='z')
ds.isel(time=ix)

<xarray.Dataset>
Dimensions:    (bnds: 2, height: 1, lat: 1, lon: 1, time: 720)
Coordinates:
  * lon        (lon) float64 -70.0
  * lat        (lat) float64 -74.0
  * height     (height) float64 0.0
  * time       (time) datetime64[ns] 2013-01-01 2013-01-01T03:00:00 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) float64 ...
    precip     (time, height, lat, lon) float32 ...
Attributes:
    CDI:           Climate Data Interface version 1.6.9 (http://mpimet.mpg.de...
    history:       Fri Aug 28 12:01:45 2015: cdo remapnn,lon=-70.0/lat=-74.0 ...
    source:        RACMO2
    institution:   Royal Netherlands Meteorological Institute
    Conventions:   CF-1.4
    Domain:        PEN055
    Experiment:    ERAIN_r507
    CreationDate:  Tue Jan 20 12:08:37 2015
    comment:       asim2cdf: cpar=precip, iwmo=61, ilvt=105, ilev=0, idh=3, i...
    title:         Total Precipitative Flux
    CDO:           Climate Data Operators version 1.6.9 (http://mpimet.

In [147]:
ds['precip'].shape #, 365*8
ds['precip'].coords
ds['precip'].dims
ds['precip'].attrs
#ds['precip'].name
#type(ds['precip'].values)


OrderedDict([('standard_name', 'precipitation_flux'),
             ('long_name', 'Total Precipitative Flux'),
             ('units', 'kg m-2 s-1'),
             ('cell_methods', 'time: 3-hr averaged values')])

In [148]:
ds2 = ds.copy()

In [74]:
ds3 = ds+ds2
ds3['precip'].values[0]
ds['precip'].values[0]

array([[[ 0.71875]]])

In [77]:
ds['precip'].mean()
#ds['precip'].sum()

<xarray.DataArray 'precip' ()>
array(0.23037213446342783)

In [150]:
ds['precip'].loc['2013-01'].sum()

<xarray.DataArray 'precip' ()>
array(30.4357967376709)

In [155]:
ds['time.month']
#ds['precip'].loc[ds['time.month']==2]
#ds['precip'].groupby(ds['time.month']).sum()
#ds['time.season']
ds['precip'].groupby(ds['time.season']).sum()
ds['precip'].sel(time=slice('2013-01-10','2013-01-12'))

<xarray.DataArray 'precip' (season: 4)>
array([ 153.572581,  146.586964,  245.563419,  126.963669])
Coordinates:
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'

In [113]:
ds.to_netcdf('/Users/leo/test.nc','w')

# Missing values

In [110]:
ds=xr.open_dataset('/Users/leo/extern/tsoi_ice.nc')

In [116]:
ds['TSOI_ICE'].shape, ds['TSOI_ICE'].dims

((1, 25, 192, 288), ('time', 'levgrnd', 'lat', 'lon'))

In [124]:
ds['TSOI_ICE'][0:10]

<xarray.DataArray 'TSOI_ICE' (time: 1, levgrnd: 25, lat: 192, lon: 288)>
array([[[[ 222.967453, ...,  222.972122],
         ..., 
         [        nan, ...,         nan]],

        ..., 
        [[ 222.804108, ...,  222.810654],
         ..., 
         [        nan, ...,         nan]]]])
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 -86.23 -85.29 -84.35 ...
  * levgrnd  (levgrnd) float32 0.01 0.04 0.09 0.16 0.26 0.4 0.58 0.8 1.06 ...
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 6.25 7.5 8.75 10.0 11.25 ...
  * time     (time) datetime64[ns] 1870-08-01
Attributes:
    long_name: soil temperature (ice landunits only)
    units: K
    cell_methods: time: mean

In [125]:
#ds['TSOI_ICE'].isnull().sum()
ds['TSOI_ICE'].notnull().sum()

<xarray.DataArray 'TSOI_ICE' ()>
array(175550)

In [128]:
climatology = ds.groupby('time.month').mean('time')

In [137]:
climatology.isel(lon=0,lat=2)

<xarray.Dataset>
Dimensions:      (hist_interval: 2, levgrnd: 25, month: 1)
Coordinates:
    lat          float64 -88.12
  * levgrnd      (levgrnd) float32 0.01 0.04 0.09 0.16 0.26 0.4 0.58 0.8 ...
    lon          float64 0.0
  * month        (month) int64 8
Dimensions without coordinates: hist_interval
Data variables:
    TSOI_ICE     (month, levgrnd) float64 223.8 223.8 223.8 223.8 223.8 ...
    time_bounds  (month, hist_interval) float64 181.0 212.0


# Links
Also see this page: http://xarray.pydata.org/en/stable/io.html#scaling-and-type-conversions

there you read that you can change the FillValue by passing an encoding dict:

    _FillValue: Values of NaN in xarray variables are remapped to this value when saved on disk. This is important when converting floating point with missing values to integers on disk, because NaN is not a valid value for integer dtypes. As a default, variables with float types are attributed a _FillValue of NaN in the output file, unless explicitly disabled with an encoding {'_FillValue': None}.



# Seasonal sums per year

In [53]:
ds = xr.open_dataset("/Users/leo/workspace/sandbox/SNOW_c2b8_UNI_fdm.003_moncat.nc")
len(ds['time']) # 20 years of monthly data = 240
ds['SNOW']

<xarray.DataArray 'SNOW' (time: 240, lndgrid: 48602)>
[11664480 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1980-02-01 1980-03-01 1980-04-01 ...
Dimensions without coordinates: lndgrid
Attributes:
    long_name:     atmospheric snow, after rain/snow repartitioning based on ...
    units:         mm/s
    cell_methods:  time: mean

In [25]:
ds['SNOW'].groupby('time.season').sum()

<xarray.DataArray 'SNOW' (season: 4)>
array([ 4.776142,  2.025796,  4.546184,  2.347533])
Coordinates:
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'

In [33]:
# WORKAROUND: apply time shift of 2 days forward 
ds['time'] -= np.timedelta64(2,'D')
ds['time']

<xarray.DataArray 'time' (time: 240)>
array(['1980-01-28T00:00:00.000000000', '1980-02-26T00:00:00.000000000',
       '1980-03-28T00:00:00.000000000', ..., '1999-10-28T00:00:00.000000000',
       '1999-11-27T00:00:00.000000000', '1999-12-28T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1980-01-28 1980-02-26 1980-03-28 ...
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bnds
    axis:           T

In [38]:
ds['SNOW'].groupby('time.year','time.DJF').sum()

<xarray.DataArray 'SNOW' (year: 20)>
array([ 0.676176,  0.686775,  0.65712 ,  0.673328,  0.68787 ,  0.692679,
        0.700808,  0.696297,  0.695693,  0.687815,  0.685581,  0.690583,
        0.657867,  0.669833,  0.677449,  0.693511,  0.69339 ,  0.676221,
        0.706451,  0.691047])
Coordinates:
  * year     (year) int64 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 ...

In [58]:
foo = ds.resample(time='Q-FEB').mean('time.year')
foo['time.month']

<xarray.DataArray 'month' (time: 81)>
array([ 2,  5,  8, 11,  2,  5,  8, 11,  2,  5,  8, 11,  2,  5,  8, 11,  2,  5,
        8, 11,  2,  5,  8, 11,  2,  5,  8, 11,  2,  5,  8, 11,  2,  5,  8, 11,
        2,  5,  8, 11,  2,  5,  8, 11,  2,  5,  8, 11,  2,  5,  8, 11,  2,  5,
        8, 11,  2,  5,  8, 11,  2,  5,  8, 11,  2,  5,  8, 11,  2,  5,  8, 11,
        2,  5,  8, 11,  2,  5,  8, 11,  2])
Coordinates:
  * time     (time) datetime64[ns] 1980-02-29 1980-05-31 1980-08-31 ...

In [55]:
foo.sel(time=foo['time.month'] == 2)

<xarray.Dataset>
Dimensions:    (bnds: 2, lndgrid: 48602, time: 21)
Coordinates:
  * time       (time) datetime64[ns] 1980-02-29 1981-02-28 1982-02-28 ...
Dimensions without coordinates: bnds, lndgrid
Data variables:
    time_bnds  (time, bnds) float64 -0.02083 31.0 334.3 365.0 699.3 730.0 ...
    SNOW       (time, lndgrid) float32 nan nan nan nan nan nan nan nan nan ...